In [2]:
import pandas as pd

### 연결기업 - 별도기업 병합

In [5]:
# 데이터 불러오기
consolidated = pd.read_csv('데이터/data/연결기업.csv', encoding='CP949')
separate = pd.read_csv('데이터/data/별도기업.csv', encoding='CP949')
kgaap = pd.read_csv('데이터/data/K-GAAP.csv', encoding='CP949')

# 모든 컬럼, 로우 출력/요약 출력
pd.set_option('display.max_columns', None)
pd.reset_option('display.max_rows', None)

In [6]:
# 데이터 프레임 행렬 수 출력

consolidated.shape
## (18021, 76)
separate.shape
## (18021, 76)
kgaap.shape
## (15706, 73)

(15706, 73)

In [7]:
# 회사 수

consolidated['회사명'].nunique()
## 2328
separate['회사명'].nunique()
## 2328
kgaap['회사명'].nunique()
## 1983

1983

In [8]:
# 데이터 프레임 결측치 합 확인
consolidated.isna().sum()

회사명                          0
거래소코드                        0
회계년도                         0
통계청 한국표준산업분류 10차(대분류)        0
총자본증가율(IFRS연결)            4221
                         ...  
종업원수증가율(IFRS)              521
총자본정상영업이익률(IFRS연결)        4221
총자본순이익률(IFRS연결)           4221
상장일                          0
상장폐지일                    16974
Length: 76, dtype: int64

In [9]:
separate.isna().sum()

회사명                            0
거래소코드                          0
회계년도                           0
통계청 한국표준산업분류 10차(대분류)          0
종업원수증가율(IFRS)                521
                           ...  
CASH FLOW 대 매출액비율(IFRS)      521
총자본정상영업이익률(IFRS)             521
종업원1인당 매출액증가율(IFRS)          521
상장일                            0
상장폐지일                      16974
Length: 76, dtype: int64

In [10]:
# 컬럼명 확인
consolidated.columns

Index(['회사명', '거래소코드', '회계년도', '통계청 한국표준산업분류 10차(대분류)', '총자본증가율(IFRS연결)',
       '정상영업이익증가율(IFRS연결)', '순이익증가율(IFRS연결)', '자기자본증가율(IFRS연결)',
       '매출액증가율(IFRS연결)', '종업원1인당 매출액증가율(IFRS)', '유형자산증가율(IFRS연결)',
       '유동자산증가율(IFRS연결)', '재고자산증가율(IFRS연결)', '매출액총이익률(IFRS연결)',
       '매출액정상영업이익률(IFRS연결)', '매출액순이익률(IFRS연결)', '자기자본정상영업이익률(IFRS연결)',
       '자기자본순이익률(IFRS연결)', '금융비용부담률(IFRS연결)', '수지비율(관계기업투자손익 제외)(IFRS연결)',
       '사내유보 대 자기자본비율(IFRS연결)', '1주당매출액(IFRS연결)', '1주당순이익(IFRS연결)',
       '1주당 CASH FLOW(IFRS연결)', '1주당순자산(IFRS연결)', '유보율(IFRS연결)',
       '금융손익비율(IFRS연결)', '매출원가 대 매출액비율(IFRS연결)', '이자부담률(IFRS연결)',
       '지급이자율(IFRS연결)', '차입금평균이자율(IFRS연결)', '자기자본구성비율(IFRS연결)', '유동비율(IFRS연결)',
       '당좌비율(IFRS연결)', '현금비율(IFRS연결)', '재고자산 대 순운전자본비율(IFRS연결)',
       '매출채권 대 매입채무비율(IFRS연결)', '부채비율(IFRS연결)', '이자보상배율(이자비용)(IFRS연결)',
       'CASH FLOW 대 부채비율(IFRS연결)', 'CASH FLOW 대 차입금비율(IFRS연결)',
       'CASH FLOW 대 총자본비율(IFRS연결)', '유동자산구성비율(IFRS연결)',
       '재고자산 대 유동자산비율(IFRS연결)', '타인자본구성비율(IFRS연결

In [11]:
# 컬럼명 불필요 문자 제거
consolidated.columns = consolidated.columns.str.replace('(', '')
consolidated.columns = consolidated.columns.str.replace(')', '')
consolidated.columns = consolidated.columns.str.replace('대분류', '')
consolidated.columns = consolidated.columns.str.replace('관계기업투자손익 제외', '')
consolidated.columns = consolidated.columns.str.replace('이자비용', '')
consolidated.columns = consolidated.columns.str.replace('IFRS', '')
consolidated.columns = consolidated.columns.str.replace('연결', '')
# consolidated.columns

<ipython-input-11-b5205b8c0f0f>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  consolidated.columns = consolidated.columns.str.replace('(', '')
<ipython-input-11-b5205b8c0f0f>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  consolidated.columns = consolidated.columns.str.replace(')', '')


In [12]:
# 컬럼명 확인
separate.columns

Index(['회사명', '거래소코드', '회계년도', '통계청 한국표준산업분류 10차(대분류)', '종업원수증가율(IFRS)',
       '매출액총이익률(IFRS)', '매출액정상영업이익률(IFRS)', '매출액순이익률(IFRS)',
       '자기자본정상영업이익률(IFRS)', '자기자본순이익률(IFRS)', '금융손익비율(IFRS)', '금융비용부담률(IFRS)',
       '수지비율(관계기업투자손익 제외)(IFRS)', '사내유보 대 자기자본비율(IFRS)', '유보율(IFRS)',
       '1주당매출액(IFRS)', '1주당순이익(IFRS)', '1주당 CASH FLOW(IFRS)', '1주당순자산(IFRS)',
       '매출원가 대 매출액비율(IFRS)', '이자부담률(IFRS)', '지급이자율(IFRS)', '차입금평균이자율(IFRS)',
       '총자본순이익률(IFRS)', '유동자산구성비율(IFRS)', '재고자산 대 유동자산비율(IFRS)',
       '자기자본배율(IFRS)', '자기자본구성비율(IFRS)', '유동비율(IFRS)', '당좌비율(IFRS)',
       '현금비율(IFRS)', '재고자산 대 순운전자본비율(IFRS)', '매출채권 대 매입채무비율(IFRS)',
       '부채비율(IFRS)', '이자보상배율(이자비용)(IFRS)', 'CASH FLOW 대 부채비율(IFRS)',
       'CASH FLOW 대 차입금비율(IFRS)', 'CASH FLOW 대 총자본비율(IFRS)', '타인자본구성비율(IFRS)',
       '매출채권비율(IFRS)', '유동부채비율(IFRS)', '단기차입금 대 총차입금비율(IFRS)', '차입금의존도(IFRS)',
       '차입금비율(IFRS)', '총자본회전률(IFRS)', '자기자본회전률(IFRS)', '타인자본회전률(IFRS)',
       '유동자산회전률(IFRS)', '당좌자산회전률(IFRS)', '재고자산회전률(IFRS)', '매출

In [13]:
# 컬럼명 불필요 문자 제거
separate.columns = separate.columns.str.replace('(', '')
separate.columns = separate.columns.str.replace(')', '')
separate.columns = separate.columns.str.replace('대분류', '')
separate.columns = separate.columns.str.replace('관계기업투자손익 제외', '')
separate.columns = separate.columns.str.replace('이자비용', '')
separate.columns = separate.columns.str.replace('IFRS', '')
separate.columns = separate.columns.str.replace('IFRS', '')
# separate.columns

<ipython-input-13-913be367b0b2>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  separate.columns = separate.columns.str.replace('(', '')
<ipython-input-13-913be367b0b2>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  separate.columns = separate.columns.str.replace(')', '')


In [14]:
# 컬럼명 재정렬
separate = separate[['회사명', '거래소코드', '회계년도', '통계청 한국표준산업분류 10차', '총자본증가율', '정상영업이익증가율',
       '순이익증가율', '자기자본증가율', '매출액증가율', '종업원1인당 매출액증가율', '유형자산증가율', '유동자산증가율',
       '재고자산증가율', '매출액총이익률', '매출액정상영업이익률', '매출액순이익률', '자기자본정상영업이익률',
       '자기자본순이익률', '금융비용부담률', '수지비율', '사내유보 대 자기자본비율', '1주당매출액', '1주당순이익',
       '1주당 CASH FLOW', '1주당순자산', '유보율', '금융손익비율', '매출원가 대 매출액비율', '이자부담률',
       '지급이자율', '차입금평균이자율', '자기자본구성비율', '유동비율', '당좌비율', '현금비율',
       '재고자산 대 순운전자본비율', '매출채권 대 매입채무비율', '부채비율', '이자보상배율', 'CASH FLOW 대 부채비율',
       'CASH FLOW 대 차입금비율', 'CASH FLOW 대 총자본비율', '유동자산구성비율', '재고자산 대 유동자산비율',
       '타인자본구성비율', '자기자본배율', '매출채권비율', '유동부채비율', '단기차입금 대 총차입금비율', '차입금의존도',
       '차입금비율', '총자본회전률', '자기자본회전률', '타인자본회전률', '유동자산회전률', '당좌자산회전률',
       '재고자산회전률', '매출채권회전률', '순운전자본회전률', '운전자본회전률', '유형자산회전율', '종업원1인당 부가가치',
       '노동장비율', '기계장비율', '자본집약도', '총자본투자효율', '설비투자효율', '기계투자효율', '부가가치율',
       '노동소득분배율', 'CASH FLOW 대 매출액비율', '종업원수증가율', '총자본정상영업이익률', '총자본순이익률',
       '상장일', '상장폐지일']]
# separate.columns

In [15]:
# 연결기업 데이터프레임의 컬럼명을 리스트로 저장
con_list = consolidated.columns.tolist()
print(con_list)

['회사명', '거래소코드', '회계년도', '통계청 한국표준산업분류 10차', '총자본증가율', '정상영업이익증가율', '순이익증가율', '자기자본증가율', '매출액증가율', '종업원1인당 매출액증가율', '유형자산증가율', '유동자산증가율', '재고자산증가율', '매출액총이익률', '매출액정상영업이익률', '매출액순이익률', '자기자본정상영업이익률', '자기자본순이익률', '금융비용부담률', '수지비율', '사내유보 대 자기자본비율', '1주당매출액', '1주당순이익', '1주당 CASH FLOW', '1주당순자산', '유보율', '금융손익비율', '매출원가 대 매출액비율', '이자부담률', '지급이자율', '차입금평균이자율', '자기자본구성비율', '유동비율', '당좌비율', '현금비율', '재고자산 대 순운전자본비율', '매출채권 대 매입채무비율', '부채비율', '이자보상배율', 'CASH FLOW 대 부채비율', 'CASH FLOW 대 차입금비율', 'CASH FLOW 대 총자본비율', '유동자산구성비율', '재고자산 대 유동자산비율', '타인자본구성비율', '자기자본배율', '매출채권비율', '유동부채비율', '단기차입금 대 총차입금비율', '차입금의존도', '차입금비율', '총자본회전률', '자기자본회전률', '타인자본회전률', '유동자산회전률', '당좌자산회전률', '재고자산회전률', '매출채권회전률', '순운전자본회전률', '운전자본회전률', '유형자산회전율', '종업원1인당 부가가치', '노동장비율', '기계장비율', '자본집약도', '총자본투자효율', '설비투자효율', '기계투자효율', '부가가치율', '노동소득분배율', 'CASH FLOW 대 매출액비율', '종업원수증가율', '총자본정상영업이익률', '총자본순이익률', '상장일', '상장폐지일']


In [16]:
# 연결기업 데이터프레임의 결측치를 개별기업 데이터로 대체
for i in con_list:
    consolidated[i] = consolidated[i].fillna(separate[i])
consolidated

,회사명,거래소코드,회계년도,통계청 한국표준산업분류 10차,총자본증가율,정상영업이익증가율,순이익증가율,자기자본증가율,매출액증가율,종업원1인당 매출액증가율,유형자산증가율,유동자산증가율,재고자산증가율,매출액총이익률,매출액정상영업이익률,매출액순이익률,자기자본정상영업이익률,자기자본순이익률,금융비용부담률,수지비율,사내유보 대 자기자본비율,1주당매출액,1주당순이익,1주당 CASH FLOW,1주당순자산,유보율,금융손익비율,매출원가 대 매출액비율,이자부담률,지급이자율,차입금평균이자율,자기자본구성비율,유동비율,당좌비율,현금비율,재고자산 대 순운전자본비율,매출채권 대 매입채무비율,부채비율,이자보상배율,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,유동자산구성비율,재고자산 대 유동자산비율,타인자본구성비율,자기자본배율,매출채권비율,유동부채비율,단기차입금 대 총차입금비율,차입금의존도,차입금비율,총자본회전률,자기자본회전률,타인자본회전률,유동자산회전률,당좌자산회전률,재고자산회전률,매출채권회전률,순운전자본회전률,운전자본회전률,유형자산회전율,종업원1인당 부가가치,노동장비율,기계장비율,자본집약도,총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,CASH FLOW 대 매출액비율,종업원수증가율,총자본정상영업이익률,총자본순이익률,상장일,상장폐지일
0,(주)CMG제약,58820,2011/12,제조업,1.98,0.00,0.00,-2.02,56.40,49.45,-5.90,17.64,-1.43,41.83,3.61,-8.50,3.51,-8.28,0.80,108.35,-245.87,406.28,-35.00,-58.60,386.51,-17.45,-12.10,58.17,1.61,7.32,8.02,66.05,200.97,165.03,15.10,35.59,602.31,51.41,3.53,-46.08,-172.40,-15.65,56.18,17.88,33.95,82.55,325.29,42.33,80.71,9.08,13.74,0.66,0.97,2.01,1.25,1.55,6.42,2.45,0.86,0.00,1.77,27.18,71.32,19.75,204.06,12.72,35.86,1.285100e+02,19.56,89.82,-24.07,4.65,2.37,-5.58,2001/08/31,NaN
1,(주)CMG제약,58820,2012/12,제조업,109.15,0.00,0.00,75.17,-8.43,-7.75,-4.12,207.08,7.73,28.34,-25.87,-40.97,-16.95,-26.84,2.53,140.55,-165.71,332.74,-138.00,-32.19,643.72,-4.31,-7.13,71.66,2.37,3.11,5.45,55.32,693.49,649.98,68.81,7.33,323.41,80.77,-7.75,-8.10,-11.83,-3.62,82.49,6.27,44.68,95.69,292.29,21.50,0.57,30.58,55.28,0.39,0.66,0.93,0.52,0.57,5.71,1.85,0.42,0.00,1.71,-0.66,69.14,18.64,425.29,0.47,2.89,1.067000e+01,1.66,1749.87,-12.71,-0.74,-9.96,-15.78,2001/08/31,NaN
2,(주)CMG제약,58820,2013/12,제조업,-2.09,0.00,0.00,55.10,25.51,23.18,16.09,-18.89,56.29,38.70,3.88,1.45,1.96,0.74,1.65,98.55,-106.33,235.52,4.16,-45.47,492.83,13.12,-1.57,61.30,2.08,80.14,12.02,87.63,694.71,610.74,394.63,14.12,282.72,14.11,0.70,-39.85,-194.73,-4.93,68.34,12.09,12.37,113.12,239.46,11.23,0.00,2.53,2.89,0.36,0.51,1.26,0.48,0.52,5.39,2.07,0.42,0.79,2.03,51.12,66.01,17.77,411.48,12.51,77.91,2.894000e+02,34.29,71.28,-13.51,1.49,1.40,0.52,2001/08/31,NaN
3,(주)CMG제약,58820,2014/12,제조업,3.86,-44.10,-46.64,0.01,11.52,-7.16,74.30,-25.93,6.37,37.07,1.94,0.69,0.90,0.32,-0.65,99.32,-106.65,262.65,1.82,8.37,447.56,13.13,0.50,62.93,-1.84,7.96,8.30,84.39,324.58,268.23,101.24,25.09,285.31,18.50,3.60,-17.73,-104.12,-2.77,48.74,17.36,15.61,113.13,274.07,17.79,0.00,2.66,3.15,0.40,0.46,2.85,0.68,0.80,4.77,1.86,0.56,0.74,1.54,43.08,113.90,32.60,354.43,12.16,37.80,1.320600e+02,31.04,79.30,-7.06,20.59,0.78,0.28,2001/08/31,NaN
4,(주)CMG제약,58820,2015/12,제조업,-2.64,0.00,0.00,-4.48,18.85,24.95,-5.37,-7.17,11.04,36.26,-9.51,-13.50,-5.37,-7.62,-0.11,113.24,-120.22,306.26,-42.00,27.30,397.60,6.02,0.11,63.74,-0.32,1272.60,10.30,82.80,296.85,235.21,29.00,31.32,254.69,20.78,-32.22,16.12,24988.82,2.77,46.47,20.77,17.20,106.02,226.14,18.91,0.00,0.01,0.01,0.47,0.56,2.88,0.99,1.22,5.22,2.10,0.75,0.60,1.48,27.67,114.25,30.43,362.77,7.63,24.21,9.092000e+01,15.96,144.10,5.80,-4.88,-4.49,-6.37,2001/08/31,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18016,흥아해운(주),3280,2016/12,운수 및 창고업,21.08,-72.25,-263.29,11.22,-1.58,-16.02,38.84,-25.08,56.39,4.49,0.71,-2.06,3.23,-9.40,3.21,100.55,6.97,8582.41,-170.00,-14.34,1879.15,205.62,-3.21,95.51,3.86,10.70,10.67,20.08,39.46,32.52,5.46,0.00,68.73,397.97,0.21,2.12,6.30,1.70,10.65,17.60,79.92,301.44,188.29,134.41,71.88,26.94,134.16,0.95,4.55,1.20,6.98,7.96,56.47,25.24,2.19,0.02,1.44,86.46,703.55,0.00,1000.62,10.98,15.81,3.509713e+09,12.66,66.49,1.96,15.87,0.67,-1.96,1976/06/29,NaN
18017,흥아해운(주),3280,2017/12,운수 및 창고업,-11.43,-322.17,0.00,-39.28,0.56,2.39,-16.50,13.29,11.18,2.09,-1.57,-8.79,-8.47,-47.52,3.67,106.15,27.74,6549.25,-581.00,-53.97,835.26,87.23,-3.67,97.91,4.10,11.43,11.69,13.77,35.76

In [17]:
# 데이터 프레임 저장
consolidated.to_csv('데이터/1차 전처리/연결별도병합.csv', encoding='CP949')

In [25]:
# 데이터 프레임 결측치 합 출력
consolidated.isna().sum()

회사명                     0
거래소코드                   0
회계년도                    0
통계청 한국표준산업분류 10차        0
총자본증가율                405
                    ...  
종업원수증가율               504
총자본정상영업이익률            405
총자본순이익률               405
상장일                     0
상장폐지일               16948
Length: 76, dtype: int64

In [26]:
# 결측치가 있는 컬럼명 출력
consolidated.columns[consolidated.isna().any()]

Index(['총자본증가율', '정상영업이익증가율', '순이익증가율', '자기자본증가율', '매출액증가율', '종업원1인당 매출액증가율',
       '유형자산증가율', '유동자산증가율', '재고자산증가율', '매출액총이익률', '매출액정상영업이익률', '매출액순이익률',
       '자기자본정상영업이익률', '자기자본순이익률', '금융비용부담률', '수지비율', '사내유보 대 자기자본비율', '1주당매출액',
       '1주당순이익', '1주당 CASH FLOW', '1주당순자산', '유보율', '금융손익비율', '매출원가 대 매출액비율',
       '이자부담률', '지급이자율', '차입금평균이자율', '자기자본구성비율', '유동비율', '당좌비율', '현금비율',
       '재고자산 대 순운전자본비율', '매출채권 대 매입채무비율', '부채비율', '이자보상배율', 'CASH FLOW 대 부채비율',
       'CASH FLOW 대 차입금비율', 'CASH FLOW 대 총자본비율', '유동자산구성비율', '재고자산 대 유동자산비율',
       '타인자본구성비율', '자기자본배율', '매출채권비율', '유동부채비율', '단기차입금 대 총차입금비율', '차입금의존도',
       '차입금비율', '총자본회전률', '자기자본회전률', '타인자본회전률', '유동자산회전률', '당좌자산회전률',
       '재고자산회전률', '매출채권회전률', '순운전자본회전률', '운전자본회전률', '유형자산회전율', '종업원1인당 부가가치',
       '노동장비율', '기계장비율', '자본집약도', '총자본투자효율', '설비투자효율', '기계투자효율', '부가가치율',
       '노동소득분배율', 'CASH FLOW 대 매출액비율', '종업원수증가율', '총자본정상영업이익률', '총자본순이익률',
       '상장폐지일'],
      dtype='object')

In [27]:
consolidated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18021 entries, 0 to 18020
Data columns (total 76 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   회사명                18021 non-null  object 
 1   거래소코드              18021 non-null  int64  
 2   회계년도               18021 non-null  object 
 3   통계청 한국표준산업분류 10차   18021 non-null  object 
 4   총자본증가율             17616 non-null  float64
 5   정상영업이익증가율          17616 non-null  float64
 6   순이익증가율             17616 non-null  float64
 7   자기자본증가율            17616 non-null  float64
 8   매출액증가율             17616 non-null  float64
 9   종업원1인당 매출액증가율      17517 non-null  float64
 10  유형자산증가율            17616 non-null  float64
 11  유동자산증가율            17616 non-null  float64
 12  재고자산증가율            17616 non-null  float64
 13  매출액총이익률            17616 non-null  float64
 14  매출액정상영업이익률         17616 non-null  float64
 15  매출액순이익률            17616 non-null  float64
 16  자기자본정상영업이익률        176

In [56]:
# 상장일, 상장폐지일을 제외하고 결측치가 있는 회사 수 출력
consolidated['회사명'][consolidated.iloc[:, 4:-2].isna().all(1)].nunique()

232

In [57]:
# 상장일, 상장폐지일을 제외하고 결측치가 있는 회사명 출력
consolidated['회사명'][consolidated.iloc[:, 4:-2].isna().all(1)].unique()

array(['(주)경동도시가스', '(주)고려시멘트', '(주)골프존', '(주)국전약품', '(주)극동자동화',
       '(주)금오하이텍', '(주)나노', '(주)나이코', '(주)나인테크', '(주)네오셈', '(주)네온테크',
       '(주)넥슨게임즈', '(주)넵튠', '(주)누보', '(주)다린', '(주)다보링크', '(주)대원모방',
       '(주)대유에이피', '(주)더블유에스아이', '(주)덴티스', '(주)드림시큐리티', '(주)디딤', '(주)디알텍',
       '(주)디와이디', '(주)라온테크', '(주)러셀', '(주)레이크머티리얼즈', '(주)로보로보', '(주)만도',
       '(주)모비스', '(주)미래생명자원', '(주)미래자원엠엘', '(주)바이오로그디바이스', '(주)베노홀딩스',
       '(주)베스파', '(주)본느', '(주)블리츠웨이', '(주)비앤에스미디어', '(주)비엔에프코퍼레이션',
       '(주)비올', '(주)비지에프리테일', '(주)비투엔', '(주)삼기', '(주)삼양사', '(주)서연이화',
       '(주)서진오토모티브', '(주)선진', '(주)세기리텍', '(주)세림비앤지', '(주)세아제강', '(주)세토피아',
       '(주)세화피앤씨', '(주)셀리드', '(주)셀바스헬스케어', '(주)스튜디오산타클로스엔터테인먼트',
       '(주)스페이스솔루션', '(주)시냅스엠', '(주)신세계아이앤씨', '(주)심텍', '(주)썸에이지',
       '(주)쎄노텍', '(주)씨엔알리서치', '(주)씨티네트웍스', '(주)아모그린텍', '(주)아이디스',
       '(주)아이비김영', '(주)아이엘사이언스', '(주)알로이스', '(주)애니플러스', '(주)애드바이오텍',
       '(주)액션스퀘어', '(주)에스에이티이엔지', '(주)에스와이이노베이션', '(주)에스와이제이',
       '(주)에이치앤아이', '(주)에이텍티앤', '(주)

In [30]:
consolidated['회사명'][consolidated.iloc[:, 4:-2].isna().all(1)]

176      (주)경동도시가스
216       (주)고려시멘트
217       (주)고려시멘트
218       (주)고려시멘트
254         (주)골프존
           ...    
17965     효성티앤씨(주)
17968      효성화학(주)
17988      휴마시스(주)
17989      휴마시스(주)
17990      휴마시스(주)
Name: 회사명, Length: 394, dtype: object

### 업종 구분  
#### 제조업, 건설업, 도소매업, 서비스업  
#### - 제조업:  
#### - 건설업:  
#### - 도소업:  
#### - 서비스업:  

In [31]:
# 컬럼명 변경
consolidated.rename(columns={"통계청 한국표준산업분류 10차" : "업종"}, inplace=True)
consolidated.head()

,회사명,거래소코드,회계년도,업종,총자본증가율,정상영업이익증가율,순이익증가율,자기자본증가율,매출액증가율,종업원1인당 매출액증가율,유형자산증가율,유동자산증가율,재고자산증가율,매출액총이익률,매출액정상영업이익률,매출액순이익률,자기자본정상영업이익률,자기자본순이익률,금융비용부담률,수지비율,사내유보 대 자기자본비율,1주당매출액,1주당순이익,1주당 CASH FLOW,1주당순자산,유보율,금융손익비율,매출원가 대 매출액비율,이자부담률,지급이자율,차입금평균이자율,자기자본구성비율,유동비율,당좌비율,현금비율,재고자산 대 순운전자본비율,매출채권 대 매입채무비율,부채비율,이자보상배율,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,유동자산구성비율,재고자산 대 유동자산비율,타인자본구성비율,자기자본배율,매출채권비율,유동부채비율,단기차입금 대 총차입금비율,차입금의존도,차입금비율,총자본회전률,자기자본회전률,타인자본회전률,유동자산회전률,당좌자산회전률,재고자산회전률,매출채권회전률,순운전자본회전률,운전자본회전률,유형자산회전율,종업원1인당 부가가치,노동장비율,기계장비율,자본집약도,총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,CASH FLOW 대 매출액비율,종업원수증가율,총자본정상영업이익률,총자본순이익률,상장일,상장폐지일
0,(주)CMG제약,58820,2011/12,제조업,1.98,0.0,0.00,-2.02,56.40,49.45,-5.90,17.64,-1.43,41.83,3.61,-8.50,3.51,-8.28,0.80,108.35,-245.87,406.28,-35.00,-58.60,386.51,-17.45,-12.10,58.17,1.61,7.32,8.02,66.05,200.97,165.03,15.10,35.59,602.31,51.41,3.53,-46.08,-172.40,-15.65,56.18,17.88,33.95,82.55,325.29,42.33,80.71,9.08,13.74,0.66,0.97,2.01,1.25,1.55,6.42,2.45,0.86,0.00,1.77,27.18,71.32,19.75,204.06,12.72,35.86,128.51,19.56,89.82,-24.07,4.65,2.37,-5.58,2001/08/31,NaN
1,(주)CMG제약,58820,2012/12,제조업,109.15,0.0,0.00,75.17,-8.43,-7.75,-4.12,207.08,7.73,28.34,-25.87,-40.97,-16.95,-26.84,2.53,140.55,-165.71,332.74,-138.00,-32.19,643.72,-4.31,-7.13,71.66,2.37,3.11,5.45,55.32,693.49,649.98,68.81,7.33,323.41,80.77,-7.75,-8.10,-11.83,-3.62,82.49,6.27,44.68,95.69,292.29,21.50,0.57,30.58,55.28,0.39,0.66,0.93,0.52,0.57,5.71,1.85,0.42,0.00,1.71,-0.66,69.14,18.64,425.29,0.47,2.89,10.67,1.66,1749.87,-12.71,-0.74,-9.96,-15.78,2001/08/31,NaN
2,(주)CMG제약,58820,2013/12,제조업,-2.09,0.0,0.00,55.10,25.51,23.18,16.09,-18.89,56.29,38.70,3.88,1.45,1.96,0.74,1.65,98.55,-106.33,235.52,4.16,-45.47,492.83,13.12,-1.57,61.30,2.08,80.14,12.02,87.63,694.71,610.74,394.63,14.12,282.72,14.11,0.70,-39.85,-194.73,-4.93,68.34,12.09,12.37,113.12,239.46,11.23,0.00,2.53,2.89,0.36,0.51,1.26,0.48,0.52,5.39,2.07,0.42,0.79,2.03,51.12,66.01,17.77,411.48,12.51,77.91,289.40,34.29,71.28,-13.51,1.49,1.40,0.52,2001/08/31,NaN
3,(주)CMG제약,58820,2014/12,제조업,3.86,-44.1,-46.64,0.01,11.52,-7.16,74.30,-25.93,6.37,37.07,1.94,0.69,0.90,0.32,-0.65,99.32,-106.65,262.65,1.82,8.37,447.56,13.13,0.50,62.93,-1.84,7.96,8.30,84.39,324.58,268.23,101.24,25.09,285.31,18.50,3.60,-17.73,-104.12,-2.77,48.74,17.36,15.61,113.13,274.07,17.79,0.00,2.66,3.15,0.40,0.46,2.85,0.68,0.80,4.77,1.86,0.56,0.74,1.54,43.08,113.90,32.60,354.43,12.16,37.80,132.06,31.04,79.30,-7.06,20.59,0.78,0.28,2001/08/31,NaN
4,(주)CMG제약,58820,2015/12,제조업,-2.64,0.0,0.00,-4.48,18.85,24.95,-5.37,-7.17,11.04,36.26,-9.51,-13.50,-5.37,-7.62,-0.11,113.24,-120.22,306.26,-42.00,27.30,397.60,6.02,0.11,63.74,-0.32,1272.60,10.30,82.80,296.85,235.21,29.00,31.32,254.69,20.78,-32.22,16.12,24988.82,2.77,46.47,20.77,17.20,106.02,226.14,18.91,0.00,0.01,0.01,0.47,0.56,2.88,0.99,1.22,5.22,2.10,0.75,0.60,1.48,27.67,114.25,30.43,362.77,7.63,24.21,90.92,15.96,144.10,5.80,-4.88,-4.49,-6.37,2001/08/31,NaN


In [32]:
consolidated['업종'].unique()

array(['제조업', '도매 및 소매업', '건설업', '전문, 과학 및 기술 서비스업', '정보통신업',
       '예술, 스포츠 및 여가관련 서비스업', '전기, 가스, 증기 및 공기조절 공급업', '금융 및 보험업',
       '운수 및 창고업', '사업시설 관리, 사업 지원 및 임대 서비스업', '교육 서비스업', '숙박 및 음식점업',
       '광업', '수도, 하수 및 폐기물 처리, 원료 재생업', '부동산업', '농업, 임업 및 어업',
       '협회 및 단체, 수리 및 기타 개인 서비스업'], dtype=object)

In [33]:
# 업종을 어떻게 합치느냐...
for i in consolidated.index:
    if consolidated.loc[i, "업종"] == "":
        consolidated.loc[i, "업종"] = ""

In [34]:
# 업종 더미변수 생성
df_dummies = pd.get_dummies(consolidated, columns=['업종'])
df_dummies

,회사명,거래소코드,회계년도,총자본증가율,정상영업이익증가율,순이익증가율,자기자본증가율,매출액증가율,종업원1인당 매출액증가율,유형자산증가율,유동자산증가율,재고자산증가율,매출액총이익률,매출액정상영업이익률,매출액순이익률,자기자본정상영업이익률,자기자본순이익률,금융비용부담률,수지비율,사내유보 대 자기자본비율,1주당매출액,1주당순이익,1주당 CASH FLOW,1주당순자산,유보율,금융손익비율,매출원가 대 매출액비율,이자부담률,지급이자율,차입금평균이자율,자기자본구성비율,유동비율,당좌비율,현금비율,재고자산 대 순운전자본비율,매출채권 대 매입채무비율,부채비율,이자보상배율,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,유동자산구성비율,재고자산 대 유동자산비율,타인자본구성비율,자기자본배율,매출채권비율,유동부채비율,단기차입금 대 총차입금비율,차입금의존도,차입금비율,총자본회전률,자기자본회전률,타인자본회전률,유동자산회전률,당좌자산회전률,재고자산회전률,매출채권회전률,순운전자본회전률,운전자본회전률,유형자산회전율,종업원1인당 부가가치,노동장비율,기계장비율,자본집약도,총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,CASH FLOW 대 매출액비율,종업원수증가율,총자본정상영업이익률,총자본순이익률,상장일,상장폐지일,업종_건설업,업종_광업,업종_교육 서비스업,업종_금융 및 보험업,"업종_농업, 임업 및 어업",업종_도매 및 소매업,업종_부동산업,"업종_사업시설 관리, 사업 지원 및 임대 서비스업","업종_수도, 하수 및 폐기물 처리, 원료 재생업",업종_숙박 및 음식점업,"업종_예술, 스포츠 및 여가관련 서비스업",업종_운수 및 창고업,"업종_전기, 가스, 증기 및 공기조절 공급업","업종_전문, 과학 및 기술 서비스업",업종_정보통신업,업종_제조업,"업종_협회 및 단체, 수리 및 기타 개인 서비스업"
0,(주)CMG제약,58820,2011/12,1.98,0.00,0.00,-2.02,56.40,49.45,-5.90,17.64,-1.43,41.83,3.61,-8.50,3.51,-8.28,0.80,108.35,-245.87,406.28,-35.00,-58.60,386.51,-17.45,-12.10,58.17,1.61,7.32,8.02,66.05,200.97,165.03,15.10,35.59,602.31,51.41,3.53,-46.08,-172.40,-15.65,56.18,17.88,33.95,82.55,325.29,42.33,80.71,9.08,13.74,0.66,0.97,2.01,1.25,1.55,6.42,2.45,0.86,0.00,1.77,27.18,71.32,19.75,204.06,12.72,35.86,1.285100e+02,19.56,89.82,-24.07,4.65,2.37,-5.58,2001/08/31,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,(주)CMG제약,58820,2012/12,109.15,0.00,0.00,75.17,-8.43,-7.75,-4.12,207.08,7.73,28.34,-25.87,-40.97,-16.95,-26.84,2.53,140.55,-165.71,332.74,-138.00,-32.19,643.72,-4.31,-7.13,71.66,2.37,3.11,5.45,55.32,693.49,649.98,68.81,7.33,323.41,80.77,-7.75,-8.10,-11.83,-3.62,82.49,6.27,44.68,95.69,292.29,21.50,0.57,30.58,55.28,0.39,0.66,0.93,0.52,0.57,5.71,1.85,0.42,0.00,1.71,-0.66,69.14,18.64,425.29,0.47,2.89,1.067000e+01,1.66,1749.87,-12.71,-0.74,-9.96,-15.78,2001/08/31,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,(주)CMG제약,58820,2013/12,-2.09,0.00,0.00,55.10,25.51,23.18,16.09,-18.89,56.29,38.70,3.88,1.45,1.96,0.74,1.65,98.55,-106.33,235.52,4.16,-45.47,492.83,13.12,-1.57,61.30,2.08,80.14,12.02,87.63,694.71,610.74,394.63,14.12,282.72,14.11,0.70,-39.85,-194.73,-4.93,68.34,12.09,12.37,113.12,239.46,11.23,0.00,2.53,2.89,0.36,0.51,1.26,0.48,0.52,5.39,2.07,0.42,0.79,2.03,51.12,66.01,17.77,411.48,12.51,77.91,2.894000e+02,34.29,71.28,-13.51,1.49,1.40,0.52,2001/08/31,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,(주)CMG제약,58820,2014/12,3.86,-44.10,-46.64,0.01,11.52,-7.16,74.30,-25.93,6.37,37.07,1.94,0.69,0.90,0.32,-0.65,99.32,-106.65,262.65,1.82,8.37,447.56,13.13,0.50,62.93,-1.84,7.96,8.30,84.39,324.58,268.23,101.24,25.09,285.31,18.50,3.60,-17.73,-104.12,-2.77,48.74,17.36,15.61,113.13,274.07,17.79,0.00,2.66,3.15,0.40,0.46,2.85,0.68,0.80,4.77,1.86,0.56,0.74,1.54,43.08,113.90,32.60,354.43,12.16,37.80,1.320600e+02,31.04,79.30,-7.06,20.59,0.78,0.28,2001/08/31,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,(주)CMG제약,58820,2015/12,-2.64,0.00,0.00,-4.48,18.85,24.95,-5.37,-7.17,11.04,36.26,-9.51,-13.50,-5.37,-7.62,-0.11,113.24,-120.22,306.26,-42.00,27.30,397.60,6.02,0.11,63.74,-0.32,1272.60,10.30,82.80,296.85,235.21,29.00,31.32,254.69,20.78,-32.22,16.12,24988.82,2.77,46.47,20.77,17.20,106.02,226.14,18.91,0.00,0.01,0.01,0.47,0.56,2.88,0.99,1.22,5.22,2.10,0.75,0.60,1.48,27.67,114.25,30.43,362.77,7.63,24.21,9.092000e+01,15.96,144.10,5.80,-4.88,-4.49,-6.37,2001/08/31,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18016,흥아해운(주),3280,2016/12,21.08,-72.25,-263.29,11.22,-1.58,-16.02,38.84,-25.08,56.39,4.49,0.71,-2.06,3.23,-9.40,3.21,100.55,6.97,8582.41,-170.00,-14.34,1879.15,205.62,-3.21,95.51,3.86,10.70,10.67,20.08,39.46,32.52,5.46

### 부실기업 컬럼 추가

In [60]:
# 부실기업 컬럼 생성 및 레이블 값 입력
consolidated["부실기업"] = ''
if consolidated["이자보상배율"] < 1:
    consolidated["부실기업"] = 1
elif consolidated["매출액증가율"] > -20 & consolidated["매출액증가율"].shift() > -20:
    consolidated["부실기업"] = 1
else:
    consolidated["부실기업"] = 0

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

파생변수 생성

데이터셋 구분

In [36]:
# 경기순환국면 확장기, 수축기
tenth_expansion = ["2011/12"]
tenth_contraction = ["2011/12", "2012/12", "2013/12"]
eleventh_expansion = ["2013/12", "2014/12", "2015/12", "2016/12", "2017/12"]
eleventh_contraction = ["2017/12", "2018/12", "2019/12", "2020/12"]
twelveth_expansion = ["2020/12", "2021/12"]

### ver.1
9개월(3분기) 이상 지속된 국면을 해당 국면으로 설정 후 & 그 외 혼재된 연도 삭제

### ver.2
6개월 이상 확장/수축이 지속된 국면을 해당 국면으로 설정

### ver.3
해당 년도 12월이 속한 국면을 그 해의 국면으로 설정